# Covid 19 Probabilities

The data analyst firm you work for has been contracted by a local Health Authority to conduct a risk assessment of Covid-19.

Your research supervisor has asked you to use the latest CDC case data to preform a Bayesian analysis to calculate the probability of death from Covid-19 among certain demographics.

# Part 1 - Download data
In order to import the data from the CDC, you'll first need to install a python library to give you access to the Socrata API used by the CDC:

    !pip install sodapy

After that, you'll need to import the latest Covid-19 case data. For an example of how to do so, visit the [Socrata CDC API Site for the Covid-19 case data](https://dev.socrata.com/foundry/data.cdc.gov/vbim-akqf/), scroll down to the section labeled `Code Snippets`, and click on the `Python Pandas` tab.

*Some things to note:*

* Since this is public data, you do not need to use an authentication token. 

* Instead of fetching 2,000 records as in the example, set your limit to 3,000,000 records. This will retrieve approximately 3/4 of the data.

* You can name your variables whatever you want. For example, when converting the data into a dataframe, you could do:


    cases = pd.DataFrame.from_records(results)

* With that many records, it will take a bit of time to download. Be careful to put the download command in a separate code cell from the rest of your analysis work, so that Colab doesn't rerun the download command everytime you make a change to your analysis.

In [ ]:
# Install sodapy and import libraries


### Initial data analysis
Once you have imported the data, perform a brief analysis to become familiar with the data.


In [ ]:
# Initial data analysis


# Part 2 - Age Demographics

The research director would like you to start by doing a simple age-based risk assessment. To do so, complete the following tasks:

* First, check to see [how many samples there are in each age bracket](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/06_calculate_statistics.html#count-number-of-records-by-category).
* Next, [filter out any records](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/03_subset_data.html#how-do-i-filter-specific-rows-from-a-dataframe) that have the values `NA` or `Unknown` for the `age_group` column, and check the counts again.  (You may find this task easier if you use a "negation" of the `isin()` function, as [discussed here](https://stackoverflow.com/questions/19960077/how-to-filter-pandas-dataframe-using-in-and-not-in-like-in-sql).


In [ ]:
# See what the values are


In [ ]:
# Filter out rows with missing age group values

# Check the updated counts


### Additional Cleanup
The research director tells you that you're going to be primarily working answer questions regarding age (`age_group`), pre-existing medical conditions (`medcond_yn`), and whether or not the patient died (`death_yn`). 

You've alread filtered out rows that were missing data for the `age_group` column, now check for missing values in the other two columns you'll be working with, then filter the data as needed, and see how many samples that leaves in the data set.

* Make sure you start with the filtered dataset, not the original.
* Make sure you pay attention to which values represent missing or unknown data for each column and filter accordingly.

In [ ]:
# Perform additional filtering


# Part 3 - Probabilities

Your research director would now like you to calculate the probability that someone in a given age demographic will die, given that they have Covid-19.

Use the [`crosstab()` function](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#cross-tabulations) to answer this qustion. You may find [this guide](https://pbpython.com/pandas-crosstab.html) useful.

* Be careful how your normalize...we want the probabilties within each age group.




In [ ]:
# Use the crosstab function to calcuate the probabilties, normalized across columns


### Visualize the data

Use Pandas' [built in plotting functions](https://pandas.pydata.org/docs/user_guide/visualization.html#bar-plots) to create a stacked bar chart of the crosstab summary.

* In order to get a visualization that makes sense, you may first need to [Transpose](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transpose.html) the summary table.

In [ ]:
# Transpose and plot the summary table as a stacked bar chart


# Part 4 - Bayesian Prediction

Your research director would like you to use a Bayesian Classifier to predict the probability that someone with Covid-19 will die or not, given their age bracket and medical status.

You'll be using [SciKit Learn's Bayesian Classifer](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) for this task.

Our use of the Bayseian Classifier in SciKit learn will have four steps. As you'll see later in the semester, some machine learning taks involve additional steps:

1. Create an instance of the classifer
2. Provide the classifier with the data it will use to build its prediction model. This is sometimes called "fitting" or "training" the model.
3. Ask the model to make a prediction by providing it with new data.
4. Interpret the prediction.

Parts of these steps will be provided for you in this exercise:

In [ ]:
# Step 1
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()

### Train the Classifer
Now we need to teach the classifier how to make a prediction based on historic data. In terms of Bayes' Theorem, we are providing the classifier with the data it needs to calculate hte conditional and joint probabilties for a particuar event.

One caveat to using most machine learning libraries is that they don't do well with textural feature data. For example, our `age_group` column contains strings indicating the patient's age bracket. The `medcond_yn` column contains strings ("Yes" or "No") indicating whether the patient had a pre-existing medical condition.

Before we can give this data to the classifier, we'll need to "encode" it unto numeric form. There are lots of ways to do this, including some built-in ways in SciKit Learn, but for now we'll use the Pandas `map()` function, which we've used before. (If you've forgotten, see the teacher's solution to [Data Exploration 02](https://byui-cse.github.io/cse450-course/module-01/exploration-02.html))

Below, the `age_group` column has been encoded into a set of numeric values. You may want to verify using the `value_counts()` function that the counts of the encoded values match up with the originals.

In [ ]:
# This code assumes your filtered dataset is stored in a variable called "filtered". 
# If it isn't, you'll need to update the code accordingly.
filtered['age_group_encoded'] = filtered['age_group'].map({
    '0 - 9 Years' : 0,
    '10 - 19 Years' : 1,
    '20 - 29 Years' : 2,
    '30 - 39 Years' : 3,
    '40 - 49 Years' : 4,
    '50 - 59 Years' : 5,
    '60 - 69 Years' : 6,
    '70 - 79 Years' : 7,
    '80+ Years': 8})

# Compare the below result to filtered['age_group'].value_counts()
filtered['age_group_encoded'].value_counts()

Create two new columns called `death_yn_encoded` and `medcond_yn_encoded` by encoding the values of the `death_yn` and `medcond_yn` columns. 

For each column, encode "Yes" as `1` and "No" as `0`.

In [ ]:
# Encode the columns


Now that you've encoded the textual features, the following code passes the data to the classifier to "train" or "fit" the model, allowing it to make predictions.

* The first parameter to the `fit()` function is called "X" and is the set of features we want to use to make our predictions. In our case, that is age group, and pre-existing medical condition status.

* The second parameter is called "Y" and is the target feature we're trying to predict. In our case, that is whether or not the patient died.

Nearly all SciKit Learn algorithms follow this pattern.

In [ ]:
# Fit the data to the model
# Again, this code assumes your filtered data is stored in a variable called
# filtered. If not, update this code accordingly
classifier.fit( filtered[ ['age_group_encoded', 'medcond_yn_encoded'] ], filtered['death_yn_encoded'])

### Make Predictions
Now that the model is trained, it's time to use it to make predictions. To do so, we can either use:

* The [`predict()`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB.predict) function, which in this case would tell us which event is most likely to occur.
* The [`predict_proba()`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB.predict_proba) function, which tells us the probability that each event would occur.

Since our research director wants to know the probabilities, we'll use the second option.

For each prediction we want to make, we would pass the classifier one sample (row) of data contanining *encoded* values for the features it knows about.

The following code shows how we could build a Pandas DataFrame to pass to the `predict_proba()` function:



In [ ]:
# Let's set the display to show floating decimal precision
# to 5 places so we avoid scientific notation
pd.options.display.float_format = '{:.5f}'.format

# Create a data frame with one sample
# Note that we build this by specifying the column name, then an 
# array of values that would go down that column.
#
# age_group_encoded | medcond_yn_encoded
# --------------------------------------
#          2        |          0        

to_predict = pd.DataFrame({
    "age_group_encoded": [2], # A 20 - 29 year old
    "medcond_yn_encoded": [0] # With no pre-existing medical conditions
})

# Wrap the results in a new Pandas data frame and display it
prediction = pd.DataFrame(classifier.predict_proba( to_predict ))

# Each row in the results is the prediction corresponding to the same row in
# our to_predict data frame. 
#
# The first column is the probability of 0 (No) for death_yn. 
# The second is the probability of 1 (Yes) for death_yn.
prediction

In [ ]:

# Create a data frame with multiple samples. 
# Note that we build these by specifying the column name, then an 
# array of values that would go down that column.
#
# age_group_encoded | medcond_yn_encoded
# --------------------------------------
#          2        |          0        
#          2        |          1        
#          5        |          1        
to_predict = pd.DataFrame({
    "age_group_encoded": [2, 2, 5], # Age groups: 20 - 29, 20 - 29, 50 - 59
    "medcond_yn_encoded": [0, 1, 1] # Pre-existing medical conditions: No, Yes, Yes
})

# Pass the data frame to the model to get a prediction,
# then wrap the results in a new Pandas data frame and display it.
prediction = pd.DataFrame(classifier.predict_proba( to_predict ))

# Each row in the results is the prediction corresponding to the same row in
# our to_predict data frame. 
#
# The first column is the probability of 0 (No) for death_yn. 
# The second is the probability of 1 (Yes) for death_yn.
prediction

For each of the following situations, use the classifier's `predict_proba()` function to predict the probability that a patient will die from Covid-19.

* A 25 year-old with no pre-existing medical conditions.
* A 25 year-old with pre-existing medical conditions.
* A 55 year-old with no pre-existing medical conditions.
* A 55 year-old with pre-existing medical conditions.
* An 80 year-old with no pre-existing medical conditions.
* An 80 year-old with pre-existing medical conditions.

Note that you can make these predictions one at a time, or all at once by passing multiple sets of data to the classifier. The results will be returned in the same order as the sample data provided.

In [ ]:
# Make the requested predictions


# Part 5: Conclusions

Based on your results, what conclusions would your provide to your research manager to pass along to the health authority? What caveats about the analysis would you include? Is this model accurate? How could you test that?

Given what you know about Bayesian models from the reading, why might the model predict that an 80 year old with no pre-existing conditions would have a 100% probability of survival?

## 🌟 Above and Beyond 🌟

The research director would like you to answer the following questions:

* How does gender affect the probability of death from Covid-19?
* If we look at likelihood of being admitted to the ICU instead of the likelihood of death, what are the probabilties across different age groups and medical statuses?
* How do the probabilities you calculated compare to the probabilties you would get if you looked only at the last three months worth of cases? You may need to review the Pandas tutorial on [How to Handle Timeseries Data](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/09_timeseries.html).
* SciKit Learn has [several different Bayesian classifiers](https://scikit-learn.org/stable/modules/naive_bayes.html). Would a different classifier produce better results? How could you test that hypothesis?